In [ ]:
"""Ignore this code cell, this is an alternate variation I was messing around with at first, start below"""
import requests
import base64

# Replace the values below with your own API key and API secret key
api_key = "R47AjI7AIbY4t9xkZLgnxCYfE"
api_secret_key = "86hsbN2nDZTvbdcXNwOaQAL4UN2p3875U5LVDXKJkKU2hXoWlT"

# Encode the API key and API secret key in base64
encoded_api_key_secret = base64.b64encode(f"{api_key}:{api_secret_key}".encode()).decode()

# Obtain a Bearer token
headers = {
    "Authorization": f"Basic {encoded_api_key_secret}",
    "Content-Type": "application/x-www-form-urlencoded;charset=UTF-8"
}
data = {
    "grant_type": "client_credentials"
}
response = requests.post("https://api.twitter.com/oauth2/token", headers=headers, data=data)

# Extract the Bearer token from the response
bearer_token = response.json()["access_token"]

# Use the Bearer token to authenticate requests to the Twitter API
headers = {
    "Authorization": f"Bearer {bearer_token}"
}

# Retrieve the most recent 100 tweets containing the word "NFT"
response = requests.get("https://api.twitter.com/2/tweets/search/recent", headers=headers, params={
    "query": "NFT",
    "max_results": 100
})

# Extract the tweets from the response
tweets = response.json()["data"]

# Print the text of each tweet
#for tweet in tweets:
 #   print(tweet["text"])

df = pd.DataFrame(tweets)

print(df)

In [2]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("BEARER_TOKEN")


def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to like.
    # You can find an ID by using the Tweet lookup endpoint
    id = "1602742534366601218"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/liking_users".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2LikingUsersPython"
    return r


def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, tweet_fields = create_url()
    json_response = connect_to_endpoint(url, tweet_fields)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()


401


Exception: Request returned an error: 401 {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}